# Stereotype Misperception Task Experiment Script
## Ryan Hutchings
### PSC 290: Scientific Programming in Python

# Introduction: What is the Stereotype Misperception Task (SMT)
The SMT is an implicit attitudes measure design to measure the extent to which participants find facial primes (in this case white and black primes) threatening. Participants are primed with white faces, black faces, or neutral face shapes before being flashed with a target face that is high or low in threat (these target faces are generated using face modeling techniques to be objectively high or low in perceived threat). 

In [1]:
from psychopy import visual, core, gui, event
import numpy as np
import pandas as pd
import random
import os
import glob
import sys

class smt_exp:
    
    def __init__(self, primes_list, targets_list, n_practice_trials, n_exp_blocks, n_exp_trials):
        
        #user-defined variables
        self.primes_list = primes_list
        self.targets_list = targets_list
        self.n_exp_blocks = n_exp_blocks
        self.n_exp_trials = n_exp_trials
        self.n_practice_trials = n_practice_trials
        
        #build the practice trials during initialization by calling the function
        self.practice_trials =  self.build_practice_trials(self.n_practice_trials, write_csv_file = False)
        
        #build the experimental trials during initialization by calling the function
        self.experiment_trials = self.build_exp_trials(self.n_exp_blocks, self.n_exp_trials, write_csv_file = False)
        
        #generate a 800x600 pixel window
        self.win = visual.Window([800,600], color="black", units='pix', fullscr= False)

        #generate a fixation cross
        self.fix = visual.TextStim(self.win, text = '+', pos = [0,0])

        #generate a prime visual stimulus
        self.prime = visual.ImageStim(self.win, image = None, pos = (0,0))

        #generate a target visual stimulus
        self.target = visual.ImageStim(self.win, image = None, pos = (0,0))

        #generate our mask visual stimulus
        self.mask = visual.ImageStim(self.win, image = 'mask.bmp', pos = (0,0))
        
        #generate low threat instructions
        self.d_instructions = visual.TextStim(self.win, text = 'low threat', pos = [-150,-150])
        
        self.k_instructions = visual.TextStim(self.win, text = 'high threat', pos = [150, -150])

        #define valid key strokes
        self.valid_response = ['d','k']
        
    def build_exp_trials(self, nblocks, ntrials, write_csv_file=False):

            #define the number of unique iterations through the list of primes and targets
            loops = ntrials/(len(self.primes_list)*len(self.targets_list))

            #preallocate list
            exp_trials = []

            #preallocate trial counter
            trial = 0

            #how many blocks in the smt experiment    
            for block in range(nblocks):

                #how many times to loop through the unique interaction of primes and targets
                for loop in range(loops):

                    #for each of the nprime types
                    for prime in self.primes_list:

                        #for each of the ntarget types
                        for target in self.targets_list:
                            
                            #selects one of many possible prime faces within a prime category
                            selected_prime = random.choice(prime)
                            
                            #selects one of many possible target faces within a target category
                            selected_target = random.choice(target)
                            
                            #concatinate current block, prime, and target into a list
                            value = [block, selected_prime, selected_target]
                            
                            #append this trial to a trial list of lists
                            exp_trials.append(value)
                            
            #unlist the built trials and melt the lists into a dataframe
            df = pd.DataFrame(exp_trials, columns = ('block', 'prime', 'target'))

            if write_csv_file:

                df.to_csv('smt_trials.csv', sep=',', index_label = 'index')

            return(df)
    
    #same underlying technique as experimental trials but with no primes (just targets)
    def build_practice_trials(self, ntrials, write_csv_file=False):

            #preallocate list
            practice_trials = []

            #preallocate trial counter
            trial = 0

            #how many blocks in the smt experiment    
            for trial in range(ntrials):
                
                #high or low threat targets
                selected_target_category = random.choice(self.targets_list)
                
                #select one of many possible threat faces in a given category
                selected_target = random.choice(selected_target_category)
                
                #concatinate trial and target image into list
                value = [trial, selected_target]
                
                #append list to list of lists
                practice_trials.append(value)
                
            #unlist and melt into dataframe
            practice_df = pd.DataFrame(practice_trials, columns = ('trial', 'target'))

            if write_csv_file:

                practice_df.to_csv('smt_trials.csv', sep=',', index_label = 'index')

            return(practice_df)
        
        
    def display_instructions(self, instruction_list):
        
        #define text stimulus for instructions to screen
        instructions = visual.TextStim(self.win, text = 'Welcome to the experiment', pos = [0,0])
        
        #define the spacebar text
        space_bar_instructions = visual.TextStim(self.win, text = '<Press the spacebar to continue>', pos = [0,-250])
        
        #number of instruction frames
        ninstr = len(instruction_list)
    
        #loop over instructions for number of instruction screens
        for curr_instr in range(ninstr):

            self.win.flip()
            core.wait(0.5)
            
            #set instructions text attribute to text from list of instructions
            instructions.setText(instruction_list[curr_instr])
            
            #draw instructions and spacebar text objects
            instructions.draw()
            space_bar_instructions.draw()
            
            self.win.flip()

            #wait for spacebar before displaying next set of instructions
            event.waitKeys(keyList = ['space'])
            
    def display_practice_trials(self):
        
        #for the number of practice trials from our previously generated df of trials        
        for practice in range(len(self.practice_trials)):
            
            #present fixation cross for 500ms
            self.fix.draw()
            self.win.flip()
            core.wait(0.5)
            self.win.flip()

            #select target to display from trials
            displayed_target = self.practice_trials['target'][practice]

            #set target attribute to displayed target
            self.target.setImage(displayed_target)
            self.target.draw()
            self.win.flip()

            #display prime for 100 ms
            core.wait(0.100)

            #draw mask to screen
            self.mask.draw()
            self.d_instructions.draw()
            self.k_instructions.draw()
            self.win.flip()

            #wait and listen for 'd' or 'k' keypress
            response = event.waitKeys(keyList=self.valid_response)
            self.win.flip()
            
            #if you press the 'q' key exit early -- for debugging
            if event.getKeys(keyList=['q']):
                
                self.win.close()
                break
                
    def display_exp_trials(self):
        
        #preallocate empty list for saving response on each trial
        response_data = []

        #for the number of trials in our previously generated experimental trials df
        for trial in range(len(self.experiment_trials)):   

            #draw fixation cross for 500ms
            self.fix.draw()
            self.win.flip()
            core.wait(0.5)

            #select prime to display from trials
            displayed_prime = self.experiment_trials['prime'][trial]

            #set prime variable to displayed image
            self.prime.setImage(displayed_prime)
            self.prime.draw()
            self.win.flip()

            #display prime for 150 ms
            core.wait(0.150)

            #select target to display from trials
            displayed_target = self.experiment_trials['target'][trial]
            
            #set target attribute to displayed target
            self.target.setImage(displayed_target)
            self.target.draw()
            self.win.flip()

            #display prime for 100 ms
            core.wait(0.100)

            #draw mask to screen
            self.mask.draw()
            self.d_instructions.draw()
            self.k_instructions.draw()
            self.win.flip()

            #wait for response
            response = event.waitKeys(keyList=self.valid_response)
            self.win.flip()
            core.wait(0.150)

            #append the response to the empty list
            response_data.append(response)

            #if you press the 'q' key exit early -- for debugging
            if event.getKeys(keyList=['q']):
                
                self.win.close()
                break
                
        #after all trials add the responses as a column to the experimental trials df
        self.experiment_trials['response'] = response_data
    
    def close_exp_win(self):
        
        #closes window to end experiment
        self.win.close()
        
    def print_exp_data(self):
        
        #prints the data collected during the experimental trials
        print(self.experiment_trials)


In [4]:
#set paths relative to current directory
path = os.getcwd()

#print path for inspection
print(path)

#direct computer toward stimulus folder
stim_path = path + '\stim'

#change directory to stimulus folder
os.chdir(stim_path)

#select images for each type of primes
black_faces = glob.glob('*bpic*')
white_faces = glob.glob('*wpic*')
face_shape = glob.glob('pn.bmp')

#prime categories
primes = [black_faces, white_faces, face_shape]

#select images for each type of target
low_threat_targets = glob.glob('*threat1*')
high_threat_targets = glob.glob('*threat5*')

#target categories
targets = [low_threat_targets, high_threat_targets]

C:\Users\rhutchin\OneDrive - UC Davis\davis\courses\python\final_project


In [5]:
#define our instruction strings -- must be unicode for psychopy parsing
instr_text_1 = (u'Welcome to this experiment!\n\n\nIt is very important that you read all instructions carefully.'
+ '\n\n\nAll information obtained in this study will be strictly confidential and will only be viewed by research personnel.')

instr_text_2 = (u'This study is about impression formation under suboptimal conditions. We ask you to quickly form impressions of various people.'
+ '\n\n\nWe will show you blurred drawings of various people, each one at a time.'
+' The drawings will be flashed very quickly and then covered by a grey pattern mask.'
+ '\n\n\nYour task is to quickly form an impression of each person: Is this person threatening?')
                
instr_text_3 = (u'In particular, we want you to indicate whether this person is more or less threatening'
+ ' than the other people you are seeing during this task.'
+'\n\n\nThus, you are asked to make a relative (not an absolute) judgment.'
+ '\n\n\nThis means that you would respond to half of the people with "less threatening" and'
+ ' to the other half of the people with "more threatening".')
                
instr_text_4 = (u'Rely on your first impression and go with your gut feelings. There are no correct or wrong answers. Respond as quickly as possible.' 
+ '\n\n\n\tIf you feel the person is less threatening than the other people press D.'
+ '\n\n\n\tIf you feel the person is more threatening than the other people press K.'
+ '\n\n\nYou can practice this task now.')

instr_text_5 = (u'Now, you will complete a variation of the impression formation task.' 
+ '\n\n\nAt the beginning of each trial, a picture of a face or a face-like shape will be presented to you immediately before the drawing of a face appears.' 
+ '\n\n\nDo NOT respond to these pictures, but do pay attention to them.'
+ '\n\n\nAfter you have finished the task, you will be asked questions about the people shown in these pictures.')
                
instr_text_6 = (u'Like in the previous trials, your main task is to quickly form an impression of the person shown in the blurred drawings'
+ '\n\n\n\tIf you fe b   el the person shown in the drawing is less threatening than the other people shown in the drawings press D.'
+ '\n\n\n\tIf you feel the person shown in the drawing is more threatening than the other people shown in the drawings press K.'
+ '\n\n\nRely on your immediate gut feeling when forming an impression!')

first_instructions = [instr_text_1, instr_text_2, instr_text_3, instr_text_4]
second_instructions = [instr_text_5, instr_text_6]

In [6]:
#run experiment
experiment = smt_exp(primes, targets, 1, 1, 6)
experiment.display_instructions(first_instructions)
experiment.display_practice_trials()
experiment.display_instructions(second_instructions)
experiment.display_exp_trials()
experiment.close_exp_win()
experiment.print_exp_data()

95.0163 	WARNING 	Monitor specification not found. Creating a temporary one...
   block       prime              target response
0      0   bpic3.bmp  fi_017_threat1.bmp      [k]
1      0  bpic27.bmp  fi_007_threat5.bmp      [k]
2      0  wpic14.bmp  fi_003_threat1.bmp      [k]
3      0   wpic8.bmp  fi_002_threat5.bmp      [k]
4      0      pn.bmp  fi_001_threat1.bmp      [k]
5      0      pn.bmp  fi_014_threat5.bmp      [k]


# Problems and future improvements
1. **Trials are not randomized within the block.** Although a random image is selected, the type of images presented are not randomized. I needed to have the same number of trials for each prime type, and I wasn't able to figure out exactly how to get the trials randomized within the block. I attempted np.shuffle() but this shuffles the primes are targets as well, so that primes end up in the target column and vice versa. I'll need to correct this before using this script. 

2. **build_exp_trial has unexpected behavior.** Because the number of loops is defined by dividing the product of all prime/target types, trials must be divisable by the product of all prime/target types. To correct this, I should assign random number to each prime type. 

3. **Position of stimuli should be in visual angle.** I'll just have to change this from visual angle, but I am not used to working in visual angles. This should reduce monitor problems. 

4. **Writing data to .txt or .csv files.** To implement fully, I'll have to use the gui to get subject information (i.e., subject number) and use this to write seperate .csv files after cleaning the data. 

5. **Read instructions in as .txt files.** It just seems to make more sense and would be easier to change our instructions. 

6. **Build in functionality to let subjects do more practice.** We usually give subjects the opportunity to do more practice if they have not gotten the hang of the task. I didn't build that functionality into this script. 

7. **Collecting reaction time.** Although reaction time is not measured for this task, it is nice to have. I can import this functionality into this script from scripts I wrote doing the Lupyan hw. 

8. **Debriefing and other little things.** Subjects need to be debriefed, and I like to link experiments to qualtrics surveys (something that I already know how to do in python and have done in the past). 